# <center/> 自定义调试体验文档

## 概述

本文将使用[快速入门](https://gitee.com/mindspore/docs/blob/master/tutorials/tutorial_code/lenet.py)作为样例，并通过构建自定义调试函数：`Callback`、`metrics`、`Print算子`、日志打印等，同时将构建的自定义调试函数添加进代码中，通过运行效果来展示具体如何使用MindSpore提供给我们的自定义调试能力，帮助快速调试训练网络。
体验过程如下：
1. 数据集准备。
2. 定义深度学习网络LeNet5。
3. 使用Callback回调函数构建StopAtTime类来控制训练停止时间。
4. 设置日志环境变量。
5. 定义模型并执行训练。
6. 执行测试。

## 数据集准备

### 数据集的下载

这里我们需要将MNIST数据集中随机取出一张图片，并增强成适合LeNet网络的数据格式（如何处理请参考[quick_start.ipynb](https://gitee.com/mindspore/docs/blob/master/tutorials/notebook/quick_start.ipynb)），训练数据集下载地址：{"<http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz>", "<http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz>"}。
<br/>数据集放在----`Jupyter工作目录+\MNIST_Data\train\`，如下图结构：

### 数据集的增强操作

下载的数据集，需要通过`mindspore.dataset`处理成适用于MindSpore框架的数据，再使用一系列框架中提供的工具进行数据增强操作来适应LeNet网络的数据处理需求。

In [1]:
import mindspore.dataset as ds
import mindspore.dataset.transforms.vision.c_transforms as CV
import mindspore.dataset.transforms.c_transforms as C
from mindspore.dataset.transforms.vision import Inter
from mindspore.common import dtype as mstype

def create_dataset(data_path, batch_size=32, repeat_size=1,
                   num_parallel_workers=1):
    """ create dataset for train or test
    Args:
        data_path (str): Data path
        batch_size (int): The number of data records in each group
        repeat_size (int): The number of replicated data records
        num_parallel_workers (int): The number of parallel workers
    """
    # define dataset
    mnist_ds = ds.MnistDataset(data_path)

    # define operation parameters
    resize_height, resize_width = 32, 32
    rescale = 1.0 / 255.0
    shift = 0.0
    rescale_nml = 1 / 0.3081
    shift_nml = -1 * 0.1307 / 0.3081

    # define map operations
    resize_op = CV.Resize((resize_height, resize_width), interpolation=Inter.LINEAR)
    rescale_nml_op = CV.Rescale(rescale_nml, shift_nml)
    rescale_op = CV.Rescale(rescale, shift)
    hwc2chw_op = CV.HWC2CHW() 
    type_cast_op = C.TypeCast(mstype.int32)

    # apply map operations on images
    mnist_ds = mnist_ds.map(input_columns="label", operations=type_cast_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=resize_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=rescale_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=rescale_nml_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=hwc2chw_op, num_parallel_workers=num_parallel_workers)

    # apply DatasetOps
    buffer_size = 10000
    mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    mnist_ds = mnist_ds.repeat(repeat_size)

    return mnist_ds

## 定义深度学习网络LeNet

针对MNIST数据集我们采用的是LeNet5网络，先对卷积函数和全连接函数初始化，然后`construct`构建神经网络。

In [2]:
from mindspore.common.initializer import TruncatedNormal
import mindspore.nn as nn
from mindspore.ops import operations as P

def conv(in_channels, out_channels, kernel_size, stride=1, padding=0):
    """Conv layer weight initial."""
    weight = weight_variable()
    return nn.Conv2d(in_channels, out_channels,
                     kernel_size=kernel_size, stride=stride, padding=padding,
                     weight_init=weight, has_bias=False, pad_mode="valid")


def fc_with_initialize(input_channels, out_channels):
    """Fc layer weight initial."""
    weight = weight_variable()
    bias = weight_variable()
    return nn.Dense(input_channels, out_channels, weight, bias)


def weight_variable():
    """Weight initial."""
    return TruncatedNormal(0.02)


class LeNet5(nn.Cell):
    """Lenet network structure."""
    def __init__(self):
        super(LeNet5, self).__init__()
        self.batch_size = 32
        self.conv1 = conv(1, 6, 5)
        self.conv2 = conv(6, 16, 5)
        self.fc1 = fc_with_initialize(16 * 5 * 5, 120)
        self.fc2 = fc_with_initialize(120, 84)
        self.fc3 = fc_with_initialize(84, 10)
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        
    def construct(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

## 构建自定义回调函数StopAtTime

使用回调函数的基类Callback，构建训练定时器`StopAtTime`，其基类（可在源码中找到位置在`/mindspore/nn/callback`）为：

```python
class Callback():
    def begin(self, run_context):
        pass
    def epoch_begin(self, run_context):
        pass
    def epoch_end(self, run_context):
        pass
    def step_begin(self, run_context): 
        pass
    def step_end(self, run_context):
        pass
    def end(self, run_context):
        pass
```

- `begin`：表示训练开始时执行。
- `epoch_begin`：表示每个epoch开始时执行。
- `epoch_end`：表示每个epoch结束时执行。
- `step_begin`：表示每个step刚开始时执行。
- `step_end`：表示每个step结束时执行。
- `end`：表示训练结束时执行。

了解上述基类的用法后，还有一个参数`run_context`，这是一个类，存储了模型训练中的各种参数，我们在这里使用`print(cb_params.list_callback)`将其放在`end`中打印（当然也可以使用`print(cb_param)`打印所有参数信息，由于参数信息太多，我们这里只选了一个参数举例），后续在执行完训练后，根据打印信息，会简单介绍`run_context`类中各参数的意义，我们开始构建训练定时器，如下：

In [3]:
from mindspore.train.callback import Callback
import time

class StopAtTime(Callback):
    def __init__(self, run_time):
        super(StopAtTime, self).__init__()
        self.run_time = run_time*60

    def begin(self, run_context):
        cb_params = run_context.original_args()
        cb_params.init_time = time.time()
    
    def step_end(self, run_context):
        cb_params = run_context.original_args()
        epoch_num = cb_params.cur_epoch_num
        step_num = cb_params.cur_step_num
        loss = cb_params.net_outputs
        cur_time = time.time()
        if (cur_time - cb_params.init_time) > self.run_time:
                print("epoch: ", epoch_num, " step: ", step_num, " loss: ", loss)
                run_context.request_stop()
    def end(self, run_context):
        cb_params = run_context.original_args()
        print(cb_params.list_callback)

## 设置日志环境变量

MindSpore采用`glog`来输出日志，我们这里将日志输出到屏幕：

`GlOG_v`：控制日志的级别，默认值为2，即WARNING级别，对应关系如下：0-DEBUG、1-INFO、2-WARNING、3-ERROR。本次设置为1。

`GLOG_logtostderr`：控制日志输出方式，设置为`1`时，日志输出到屏幕；值设置为`0`时，日志输出到文件。设置输出屏幕时，日志部分的信息会显示成红色，设置成输出到文件时，会在`GLOG_log_dir`路径下生成`mindspore.log`文件。

> 更多设置请参考官网：<https://www.mindspore.cn/tutorial/zh-CN/master/advanced_use/customized_debugging_information.html>

In [4]:
import os
from mindspore import log as logger

os.environ['GLOG_v'] = '1'
os.environ['GLOG_logtostderr'] = '1'
os.environ['GLOG_log_dir'] = 'D:/' if os.name=="nt" else '/var/log/mindspore'
os.environ['logger_maxBytes'] = '5242880'
os.environ['logger_backupCount'] = '10'
print(logger.get_log_config())

{'GLOG_v': '1', 'GLOG_logtostderr': '1'}


打印信息为`GLOG_v`的等级：`INFO`级别。

输出方式`GLOG_logtostderr`：`1`表示屏幕输出。

## 定义网络模型并执行训练

### 定义网络模型

此过程中先将之前生成的模型文件`.ckpt`和`.meta`的数据删除，并将模型需要用到的参数配置到`Model`。

In [5]:
from mindspore import context
from mindspore.train import Model
from mindspore.nn.metrics import Accuracy
from mindspore.nn.loss import SoftmaxCrossEntropyWithLogits
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor

# clean files
if os.name == "nt":
    os.system('del/f/s/q *.ckpt *.meta')
else:
    os.system('rm -f *.ckpt *.meta *.pb')

context.set_context(mode=context.GRAPH_MODE, device_target="CPU")
lr = 0.01
momentum = 0.9 
epoch_size = 3
train_data_path = "./MNIST_Data/train"
eval_data_path = "./MNIST_Data/train"

net_loss = SoftmaxCrossEntropyWithLogits(is_grad=False, sparse=True, reduction='mean')
repeat_size = epoch_size
network = LeNet5()

metrics = {
    'accuracy': nn.Accuracy(),
    'loss': nn.Loss(),
    'precision': nn.Precision(),
    'recall': nn.Recall(),
    'f1_score': nn.F1()
    }
net_opt = nn.Momentum(network.trainable_params(), lr, momentum)

config_ck = CheckpointConfig(save_checkpoint_steps=1875, keep_checkpoint_max=10)

ckpoint_cb = ModelCheckpoint(prefix="checkpoint_lenet", config=config_ck)

model = Model(network, net_loss, net_opt, metrics=metrics)

### 执行训练

在构建训练网络中，给`model.train`传入了三个回调函数，分别是`ckpoint_cb`，`LossMonitor`，`stop_cb`；其分别代表如下：

`ckpoint_cb`：即是`ModelCheckpoint`，设置模型保存的回调函数。

`LossMonitor`：loss值监视器，打印训练过程每步的loss值。

`stop_cb`：即是`StopAtTime`，上面刚构建的训练定时器。

我们将训练定时器`StopAtTime`设置成18秒，即`run_time=0.3`。

In [6]:
print("============== Starting Training ==============")
ds_train = create_dataset(train_data_path, repeat_size = repeat_size)
stop_cb = StopAtTime(run_time=0.3)
model.train(epoch_size, ds_train, callbacks=[ckpoint_cb, LossMonitor(375), stop_cb], dataset_sink_mode=False)

============== Starting Training ==============


[INFO] ME(10004:11540,MainProcess):2020-07-22-16:52:22.904.779 [mindspore\train\serialization.py:308] Execute save the graph process.


epoch: 1 step 375, loss is 2.3015153408050537
epoch: 1 step 750, loss is 2.2981557846069336
epoch: 1 step 1125, loss is 2.304901361465454
epoch: 1 step 1500, loss is 0.27651622891426086


[INFO] ME(10004:11540,MainProcess):2020-07-22-16:52:33.315.965 [mindspore\train\serialization.py:119] Execute save checkpoint process.
[INFO] ME(10004:11540,MainProcess):2020-07-22-16:52:33.325.978 [mindspore\train\serialization.py:147] Save checkpoint process finish.


epoch: 1 step 1875, loss is 0.263612300157547
Epoch time: 11051.060, per step time: 5.894, avg loss: 1.702
************************************************************
epoch: 2 step 375, loss is 0.22589832544326782
epoch: 2 step 750, loss is 0.12003941088914871


[INFO] ME(10004:11540,MainProcess):2020-07-22-16:52:40.282.209 [mindspore\train\serialization.py:119] Execute save checkpoint process.
[INFO] ME(10004:11540,MainProcess):2020-07-22-16:52:40.297.275 [mindspore\train\serialization.py:147] Save checkpoint process finish.


epoch:  2  step:  2927  loss:  0.26415202
Epoch time: 6953.909, per step time: 3.709, avg loss: 0.130
************************************************************
[<mindspore.train.callback._checkpoint.ModelCheckpoint object at 0x000001ACCDDA8688>, <mindspore.train.callback._loss_monitor.LossMonitor object at 0x000001ACBCF3AE08>, <__main__.StopAtTime object at 0x000001ACBD016148>]


以上打印信息中，主要分为两部分：
- 日志信息部分：
    - `[INFO]`部分信息即为日志输出的信息，由于没有Warning信息，目前主要记录的是训练的几个重要步骤。
    
- 回调函数信息部分：
    - `LossMonitor`：每步的loss值。
    - `StopAtTime`：在每个epoch结束及训练时间结束时，打印当前epoch的训练总时间(单位为毫秒)，每步训练花费的时间以及平均loss值，另外在训练结束时还打印了`run_context.list_callback`的信息，这条信息表示本次训练过程中使用的回调函数；另外`run_conext.original_args`中还包含以下参数：
        - `train_network`：网络的各类参数。
        - `epoch_num`：训练的epoch数。
        - `batch_num`：一个epoch的step数。
        - `mode`：MODEL的模式。
        - `loss_fn`：使用的损失函数。
        - `optimizer`：使用的优化器。
        - `parallel_mode`：并行模式。
        - `device_number`：训练卡的数量。
        - `train_dataset`：训练的数据集。
        - `list_callback`：使用的回调函数。
        - `train_dataset_element`：打印当前batch的数据集。
        - `cur_step_num`：当前训练的step数。
        - `cur_epoch_num`：当前的epoch。
        - `net_outputs`：网络返回值。

   几乎在训练中的所有重要数据，都可以从Callback中取得，所以Callback也是在自定义调试中比较常用的功能。

## 执行测试

测试网络中我们的自定义函数`metrics`将在`model.eval`中被调用，除了模型的预测正确率外`recall`，`F1`等不同的检验标准下的预测正确率也会打印出来：

In [7]:
print("============== Starting Testing ==============")
ds_eval = create_dataset(eval_data_path, repeat_size=repeat_size)
acc = model.eval(ds_eval,dataset_sink_mode = False)
print("============== Accuracy:{} ==============".format(acc))

============== Starting Testing ==============
============== Accuracy:{'accuracy': 0.9712666666666666, 'loss': 0.0918103571044902, 'precision': array([0.979007  , 0.9815034 , 0.9695254 , 0.99006449, 0.97207177,
       0.9750469 , 0.97660037, 0.9832609 , 0.91311292, 0.97466828]), 'recall': array([0.99206483, 0.98383269, 0.97717355, 0.92643941, 0.98305375,
       0.95867921, 0.9873268 , 0.97509976, 0.97709793, 0.95074802]), 'f1_score': array([0.98549266, 0.98266667, 0.97333445, 0.95719582, 0.97753191,
       0.96679379, 0.98193429, 0.97916333, 0.94402246, 0.96255956])} ==============


`[INFO]`部分为日志信息。

`Accuracy`部分的信息即为`metric`控制输出的信息，模型的预测值正确率和其他标准下验证（0-9）的正确率值，至于不同的验证标准计算方法，大家可以去官网搜索`mindspore.nn`查找，这里就不多介绍了。

## 总结

我们使用了MNIST数据集，通过LeNet5神经网络进行训练，将自定义调试函数结合进其代码中进行调试，展示了使用方法和部分功能，并在过程中展示了训练过程中我们能够通过自定义调试函数输出的数据，来更好的认识自定义调试函数的方便性，以上就是本次的体验内容。